In [ ]:
# NB: Unfinished prototype

In [ ]:
from google.cloud import bigquery

# noinspection PyTypeChecker
client = bigquery.Client(project="jupyter-local-project")

In [ ]:
table_id = "the-psf.pypi.distribution_metadata"
table = client.get_table(table_id)
table.schema

In [ ]:
from pathlib import Path
from tqdm import tqdm

from datetime import datetime
import json


class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        return json.JSONEncoder.default(self, o)


# Save a lot of disk space
selected_fields = [field for field in table.schema if field.name != "description"]

table_id = "the-psf.pypi.distribution_metadata"
table = client.get_table(table_id)
with Path("pypi_all.ndjson").open("w") as fp:
    rows_iter = client.list_rows(table_id, selected_fields=selected_fields)
    for row in tqdm(rows_iter, total=table.num_rows):
        fp.write(json.dumps(dict(row), cls=DateTimeEncoder))
        fp.write("\n")